In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('./data/data.csv')

In [5]:
df.dtypes

Unnamed: 0                       int64
encounter_id                     int64
hospital_id                      int64
age                            float64
bmi                            float64
                                ...   
immunosuppression                int64
leukemia                         int64
lymphoma                         int64
solid_tumor_with_metastasis      int64
diabetes_mellitus                int64
Length: 181, dtype: object

In [27]:
column_list = df.columns
column_list

Index(['Unnamed: 0', 'encounter_id', 'hospital_id', 'age', 'bmi',
       'elective_surgery', 'ethnicity', 'gender', 'height',
       'hospital_admit_source',
       ...
       'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min', 'aids', 'cirrhosis',
       'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis', 'diabetes_mellitus'],
      dtype='object', length=181)

In [6]:
df['diabetes_mellitus'].value_counts()

0    102006
1     28151
Name: diabetes_mellitus, dtype: int64

In [8]:
percent_target = 28151/df.shape[0]

0.21628494817796967

In [20]:
percent_missing = df.isna().sum().sum()/df.size

0.3403536409088947

# FUNCTIONS

In [59]:
def percent_na_df(df,feature_list):
    
    '''
    This function takes in a dataframe and feature list
    (1) calculates the percentage of NAN per feature
    
    Returns a dataframe showing percent NAN per feature
    '''
    
    num_na = []
    col_list = []
    for column in df[feature_list].columns:
        col_list.append(column)
        num_na.append(round(df[column].isna().sum()/df.shape[0]*100,2))

    zipped = list(zip(col_list,num_na))
    zorted = sorted(zipped,key=lambda x:x[1],reverse=True)
    
    zorted_vals = [[val[1]] for val in zorted]
    zorted_feats = [[feat[0]]for feat in zorted]
    
    nans_ = pd.DataFrame(zorted_vals,columns=['percent_na'])
    feats_ = pd.DataFrame(zorted_feats,columns=['features'])
    nans_df = feats_.merge(nans_,left_index=True,right_index=True)
    nans_df_sorted = nans_df.sort_values(['percent_na'],ascending=True)
    return nans_df_sorted


def select_correlated(df,feature_list):
       
    '''
    This function takes in a dataframe
    (1) calculates the correlation of each feature with the target
    (2) creates a dataframe of features, correlations, and percent NA
    (3) drops the features where more than 58% of the data is NA
    (4) drops the features that are not very correlated with target
    
    Returns the dataframe
    '''
    
    nans_df = percent_na_df(df,feature_list)
    corr_ = df.corr()[['diabetes_mellitus']].reset_index()
    corr_nans = nans_df.merge(corr_,left_on='features',right_on='index')
    corr_nans.drop(columns='index',axis=1,inplace=True)
    
    corr_sort = corr_nans.sort_values(['diabetes_mellitus'],ascending=False)
    corr_sort_drop = corr_sort[corr_sort['percent_na'] < 58].copy()
    corr_sort_drop_2 = corr_sort_drop[(corr_sort_drop['diabetes_mellitus'] > .03) | \
                                      (corr_sort_drop['diabetes_mellitus'] < -.03)]
    
    return (corr_sort_drop_2)

    
def correlated_features(df,feature_list):
    '''
    This function returns the feature list from the above function
    '''
    
    corr_sort_drop_2 = select_correlated(df,feature_list)  
    return (corr_sort_drop_2['features'])


def na_feature_list(df,feature_list):
    
    '''This function returns a subset of features -- the features that contain NAN values
    --- from the correlated features list above'''
    
    nans_df = percent_na_df(df,feature_list)
    nans_df_drop = nans_df[nans_df['percent_na'] != 0]
    
    return nans_df_drop['features']

def no_nulls_features(df,feature_list):
    
    '''This function returns a subset of features -- the features that DO NOT contain NAN values
    --- from the full list of continuous features'''

    df_ = percent_na_df(df,feature_list)
    nonull_df = df_[df_['percent_na'] == 0]
    
    return nonull_df['features']

# SUPER BASIC

In [29]:
continuous_features = []
categorical_features = []
def type_groups(df):
    for column in df.columns:
        if df[column].dtype == 'int64' or df[column].dtype == 'float64':
            continuous_features.append(column)
        elif df[column].dtype == 'object':
            categorical_features.append(column)
        else:
            pass
    return

In [30]:
type_groups(df)

In [31]:
# isolate continuous features minus the irrelevant 'Unnamed: 0', 'encounter_id', 'hospital_id'

continuous_features = continuous_features[3:].copy()

In [32]:
# insert target to our categorical_features list
# for correlation calculation later

categorical_features.append('diabetes_mellitus')

In [33]:
# set up DF with continuous variables `and categorical variables to handle separately

df_cont = df[continuous_features]
df_cat = df[categorical_features]

In [34]:
df_cat.to_csv('./data/categorical_df.csv',index=False)

In [35]:
feature_list = correlated_features(df_cont,continuous_features)[:15]
feature_list

['diabetes_mellitus',
 'd1_glucose_max',
 'glucose_apache',
 'h1_glucose_max',
 'h1_glucose_min',
 'bmi',
 'weight',
 'd1_bun_max',
 'bun_apache',
 'd1_bun_min',
 'd1_glucose_min',
 'd1_creatinine_max',
 'd1_creatinine_min',
 'creatinine_apache',
 'arf_apache']

In [36]:
first_pass = df_cont[feature_list]
df_fillna = first_pass.fillna(first_pass.median())
df_fillna

,diabetes_mellitus,d1_glucose_max,glucose_apache,h1_glucose_max,h1_glucose_min,bmi,weight,d1_bun_max,bun_apache,d1_bun_min,d1_glucose_min,d1_creatinine_max,d1_creatinine_min,creatinine_apache,arf_apache
0,1,168.0,168.0,140.0,134.0,22.732803,73.9,31.0,31.0,30.0,109.0,2.51,2.23,2.51,0
1,1,145.0,145.0,145.0,143.0,27.421875,70.2,11.0,9.0,9.0,128.0,0.71,0.56,0.56,0
2,0,150.0,133.0,140.0,134.0,31.952749,95.3,19.0,19.0,17.0,108.0,1.00,0.94,0.97,0
3,0,185.0,185.0,140.0,134.0,22.635548,61.7,19.0,19.0,17.0,88.0,1.00,0.94,0.97,0
4,0,150.0,133.0,140.0,134.0,27.564749,80.0,19.0,19.0,17.0,108.0,1.00,0.94,0.97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130152,0,150.0,133.0,140.0,134.0,29.287256,90.0,19.0,19.0,17.0,108.0,1.00,0.94,0.97,0
130153,0,139.0,139.0,140.0,134.0,29.653433,78.4,32.0,32.0,32.0,139.0,1.07,1.07,1.07,0
130154,1,346.0,346.0,140.0,134.0,32.265371,102.0,31.0,30.0,30.0,55.0,2.65,2.41,2.43,0
130155,0,156.0,137.0,140.0,134.0,24.408579,83.9,45.0,19.0,45.0,137.0,2.21,2.21,0.97,0


In [37]:
X = df_fillna.iloc[:,1:]
y = df_fillna['diabetes_mellitus']

X_train, X_hold, y_train, y_hold = train_test_split(X, y, test_size=0.20, random_state=18)

X_train.to_csv('./data/processed_first_X_15.csv',index=False)
y_train.to_csv('./data/processed_first_y_15.csv',index=False)

# DUMMY CATEGORICAL VARIABLE HANDLING

In [38]:
# dummify

cat_dummies = pd.get_dummies(df_cat,columns=['ethnicity','gender','hospital_admit_source',
                                             'icu_admit_source','icu_stay_type','icu_type'],
                             dummy_na=True)

In [39]:
# combine ethnicity "other" and ethnicity "na" to one category
# drop original
# drop icu stay and icu type nan

cat_dummies['ethnicity_other_na'] = cat_dummies['ethnicity_Other/Unknown'] + \
                                                                cat_dummies['ethnicity_nan']
cat_dummies.drop(columns=['ethnicity_Other/Unknown','ethnicity_nan',
                          'icu_stay_type_nan','icu_type_nan'],inplace=True,axis=1)

In [40]:
# drop some other categories

cat_dummies.drop(columns=['ethnicity_Native American','gender_nan',
                          'icu_type_CTICU','icu_stay_type_readmit'],axis=1,inplace=True)

# APPLY FUNCTIONS

In [42]:
# continuous variable correlation DF

select_correlated(df_cont,continuous_features)

,features,percent_na,diabetes_mellitus
0,diabetes_mellitus,0.00,1.000000
63,d1_glucose_max,6.33,0.400742
78,glucose_apache,11.29,0.354359
102,h1_glucose_max,57.68,0.316847
101,h1_glucose_min,57.68,0.304520
47,bmi,3.45,0.169043
44,weight,2.66,0.155517
77,d1_bun_max,10.55,0.146990
93,bun_apache,19.52,0.145241
76,d1_bun_min,10.55,0.137304


In [43]:
# continuous variable correlated features list

correlated_features = correlated_features(df_cont,continuous_features)

In [44]:
# percent NAN of selected correlated feature (37) DF

percent_na_df(df_cont,correlated_features)

,features,percent_na
36,arf_apache,0.00
35,diabetes_mellitus,0.00
33,d1_sysbp_max,0.21
34,d1_diasbp_min,0.21
32,d1_sysbp_noninvasive_max,1.25
31,d1_diasbp_noninvasive_min,1.26
30,weight,2.66
29,bmi,3.45
28,age,3.83
27,h1_sysbp_max,4.24


In [45]:
# features with NAN in our selected correlated features

na_features = na_feature_list(df_cont,correlated_features)

In [46]:
na_features #35 features -- REMINDER arf_apache not in list 

33                 d1_sysbp_max
34                d1_diasbp_min
32     d1_sysbp_noninvasive_max
31    d1_diasbp_noninvasive_min
30                       weight
29                          bmi
28                          age
27                 h1_sysbp_max
26                h1_diasbp_min
25                h1_diasbp_max
24               d1_glucose_min
23               d1_glucose_max
22     h1_sysbp_noninvasive_max
21    h1_diasbp_noninvasive_min
20    h1_diasbp_noninvasive_max
19             d1_potassium_min
18             d1_potassium_max
16            d1_creatinine_min
15            d1_creatinine_max
17                d1_sodium_min
14                   d1_bun_min
13                   d1_bun_max
12               glucose_apache
11            d1_hematocrit_max
10            d1_hematocrit_min
8             d1_hemaglobin_min
9             d1_hemaglobin_max
7                d1_calcium_max
6                   d1_hco3_min
5                 sodium_apache
4             creatinine_apache
3       

In [51]:
# features with NO NAN values that we will use to train model

notnull_features = no_nulls_features(df_cont,continuous_features)[1:] #remove target
notnull_features

156               elective_surgery
157                         icu_id
158               pre_icu_los_days
159             readmission_status
160          apache_post_operative
161                     arf_apache
162               intubated_apache
170    solid_tumor_with_metastasis
164                           aids
169                       lymphoma
168                       leukemia
163              ventilated_apache
166                hepatic_failure
165                      cirrhosis
167              immunosuppression
Name: features, dtype: object

# IMPUTE NAN WITH KNN

In [47]:
from sklearn.neighbors import KNeighborsClassifier

In [48]:
df_copy = df_cont.copy()

In [52]:
def impute_these_NAN(df,notnull_features,na_features):
    copy = df.copy()
    
    for feature in na_features:
        df_ = copy[copy[feature].notnull()]
        
        X = df_[notnull_features]
        y = df_[feature].astype(int)
#        print('\n\nFEATURE: ',feature)
#        print('X-matrix SHAPE: ',X.shape)
#        print('y SHAPE: ',y.shape)
    
        knn = KNeighborsClassifier(3,weights='distance')
        model = knn.fit(X,y)

        missing = copy[feature].isnull()
        df_missing = pd.DataFrame(copy[notnull_features][missing])
        
        impute = model.predict(df_missing)
        
        gen_alt = (alt for alt in impute)
        for i, value in enumerate(copy[feature]):
            if not pd.isnull(value): continue
            try:
                copy[feature].iloc[i] = next(gen_alt)
            except StopIteration:
                break
                
#        print(copy[feature].isnull().value_counts())
    return copy[na_features]

In [ ]:
impute_df = impute_these_NAN(df_copy,notnull_features,na_features)

In [54]:
# sanity check ...did we actually impute?
impute_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130157 entries, 0 to 130156
Data columns (total 35 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   d1_sysbp_max               130157 non-null  float64
 1   d1_diasbp_min              130157 non-null  float64
 2   d1_sysbp_noninvasive_max   130157 non-null  float64
 3   d1_diasbp_noninvasive_min  130157 non-null  float64
 4   weight                     130157 non-null  float64
 5   bmi                        130157 non-null  float64
 6   age                        130157 non-null  float64
 7   h1_sysbp_max               130157 non-null  float64
 8   h1_diasbp_min              130157 non-null  float64
 9   h1_diasbp_max              130157 non-null  float64
 10  d1_glucose_min             130157 non-null  float64
 11  d1_glucose_max             130157 non-null  float64
 12  h1_sysbp_noninvasive_max   130157 non-null  float64
 13  h1_diasbp_noninvasive_min  13

In [57]:
select_correlated(cat_dummies,cat_dummies.columns)

,features,percent_na,diabetes_mellitus
0,diabetes_mellitus,0.0,1.000000
21,ethnicity_African American,0.0,0.038974
16,icu_type_Neuro ICU,0.0,-0.041133
23,ethnicity_Caucasian,0.0,-0.047657


In [61]:
# narrow down categorical features

cat_correlated_features = correlated_features(cat_dummies,cat_dummies.columns)
cat_correlated_features
df_cat_select = cat_dummies[cat_correlated_features]

In [62]:
df_all_but_one = df_cat_select.merge(impute_df,how='inner',left_index=True,right_index=True)
df_all = df_all_but_one.merge(df_cont['arf_apache'],left_index=True,right_index=True)

# FOR MODELING

In [ ]:
df_all.to_csv('./data/processed_all.csv',index=False)

In [ ]:
X = df_all.iloc[:,1:]
y = df_all['diabetes_mellitus']

X_train, X_hold, y_train, y_hold = train_test_split(X, y, test_size=0.20, random_state=18)

X_train.to_csv('./data/processed_second_X.csv',index=False)
y_train.to_csv('./data/processed_second_y.csv',index=False)

In [ ]:
X_hold.to_csv('./data/processed_HOLDOUT_X.csv',index=False)
y_hold.to_csv('./data/processed_HOLDOUT_y.csv',index=False)

# TABLEAU

In [ ]:
df_tableau_= df_cat.merge(impute_df,how='inner',left_index=True,right_index=True)
df_tableau = df_tableau_.merge(df_cont['arf_apache'],left_index=True, right_index=True)

In [ ]:
df_tableau.to_csv('./data/for_tableau.csv',index=False)